In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!pwd

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content


In [2]:
import os
os.chdir('/content/drive/My Drive/isic/')
!ls

ae_datagenerator.py	     models		     train_npairs.ipynb
c2ae.py			     npair_generator.py      tsne.ipynb
classifier_datagenerator.py  __pycache__	     utils.py
data			     train_classifier.ipynb
model.py		     train_classifier.py


In [3]:
!pip install keras==2.1.5
%tensorflow_version 1.x
import keras
keras.__version__

     |████████████████████████████████| 337kB 13.3MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


Using TensorFlow backend.


'2.1.5'

In [4]:
!pip install efficientnet

In [5]:
from ae_datagenerator import get_encoder
from tensorflow.contrib.losses import metric_learning
from npair_generator import generator
from utils import EFNB4_INPUT_SHAPE
from keras.optimizers import Adam
from keras.layers import Input, Concatenate
from keras.models import Model

In [0]:
MODEL_NAME = 'efnb4'

In [0]:
def np_loss(batch_size):
    def loss(y_true,y_pred):
        labels = y_pred[:int(batch_size/2),-1]
        embeddings_anchor = y_pred[:int(batch_size/2),:-1]
        embeddings_positive = y_pred[int(batch_size/2):,:-1]
        return metric_learning.npairs_loss(labels=labels,
                           embeddings_anchor=embeddings_anchor,
                           embeddings_positive=embeddings_positive)
    return loss

In [0]:
batch_size = 4

In [0]:
gen = generator(batch_size=batch_size,
                model_name=MODEL_NAME,
                mode='train',image_size=EFNB4_INPUT_SHAPE[:-1],
                preprocess=True,
                onehotencode=False)

In [0]:
encoder = get_encoder(MODEL_NAME)
encoder.trainable = True

In [10]:
label_input = Input(shape=(1,))
image_input = Input(shape=EFNB4_INPUT_SHAPE)
embedding = encoder(image_input)
out = Concatenate()([embedding,label_input])

model = Model([image_input,label_input],out)











Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

78856192/78856072 [==============================] - 7s 0us/step


In [11]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 380, 380, 3)  0                                            
__________________________________________________________________________________________________
Encoder (Model)                 (None, 1024)         19509848    input_5[0][0]                    
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 1025)         0           Encoder[2][0]                    
                                                                 input_4[0][0]                    
Total para

In [0]:
model.compile(Adam(0.001),np_loss(batch_size))

In [18]:
model.fit_generator(generator=gen,
                    steps_per_epoch=1,
                    epochs=1)

Epoch 1/1
1/1 [==============================] - 41s 41s/step - loss: 0.0596
